In [32]:
import os.path
import string
from os import listdir
import numpy as np
import pandas as pd


#Folder containing the labeled snippet subfolders for training the classifier
#snippetFolder = "/home/abc/ADM/Train_Data" #for Chandra
snippetFolder = "Train_Data" #for Dan

#folder containing unlabeled snippets
#testSnippetFolder = "/home/abc/ADM/Test_Data" #for Chandra
testSnippetFolder = "Test_Data" #for Dan

#folderToSaveResultsFile = "/home/abc/ADM/Results" #for Chandra
folderToSaveResultsFile = "Results" #for Dan

#list of all folders in the training data
folder_list = [f for f in listdir(snippetFolder) if not f.startswith('.')]

puncs = string.punctuation

X = []
Y = []
for folder in folder_list:
    files_list = [f for f in listdir(snippetFolder + "/" + folder) if not f.startswith('.')]
    for file in files_list:
        filePath = snippetFolder + "/" + folder + "/" + file
        f = open(filePath,'r',encoding='utf-8')
        data = f.readlines()
        textSnippet = ''.join(data)
        textSnippet = textSnippet.replace("\n","")
        for p in puncs:
            if p in textSnippet:
                textSnippet = textSnippet.replace(p,"")
        X.append(textSnippet)
        Y.append(folder)
    
print(len(X))
print(len(Y))

#maximum number of features you want to consider to build the model
max_features = None #None means no limit

#which n-gram model to use? if low is 2 and high is 5, we will consider all 2-grams to 5-grams
low = 2 
high = 2

#word occurences
min_freq = 6 #np.ceil(len(X) / 10)
max_freq = np.ceil(len(X) / 1.5)

print(min_freq)
print(max_freq)

#number of folds for cross-validation
cv=5



#from sklearn.feature_extraction import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score

"""
tfidf = TfidfVectorizer(input='content',encoding='utf-8',decode_error='ignore',strip_accents='ascii',\
                        ngram_range=(2,2),stop_words='english',max_df=80,min_df=6,max_features=None,\
                        norm='l1')
"""
tfidf = TfidfVectorizer(input='content', encoding='utf-8', decode_error='ignore', strip_accents='ascii',\
                        ngram_range=(low,high), stop_words='english', max_features=max_features, norm='l1',\
                        max_df=max_freq, min_df=min_freq)
train = tfidf.fit_transform(X,Y)
print(len(tfidf.get_feature_names()))
print(tfidf.get_feature_names())
print(train.shape)
#print("\nWords Removed : \n",tfidf.stop_words_)
#print(tfidf.build_analyzer())


164
164
6
110.0
19
['architecture chicago', 'architecture history', 'armour institute', 'art institute', 'art school', 'building chicago', 'carl condit', 'chicago school', 'chicago university', 'civics philanthropy', 'condit chicago', 'institute armour', 'institute technology', 'new york', 'school architecture', 'school chicago', 'school civics', 'university chicago', 'university illinois']
(164, 19)


In [33]:
#DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

criterion = ["gini","entropy"]
splitter = ["best", "random"]


clf_dt = DecisionTreeClassifier(criterion="gini", splitter="best", max_features=None, max_depth=None, \
                                min_samples_split=2, min_samples_leaf=1, max_leaf_nodes=None, \
                                class_weight="balanced", random_state=123, presort=False)

dt_acc = cross_val_score(clf_dt,train,Y,scoring="accuracy",cv=3)
#print(acc)
print(np.mean(dt_acc))

clf_dt.fit(train, Y)

#print(len(clf_dt.feature_importances_))
#print(clf_dt.feature_importances_)
#print(clf_dt.tree_)
#tree.export_graphviz(clf_dt,out_file='tree.dot')
df = pd.DataFrame(data= {"Features" : tfidf.get_feature_names(), "weights" : clf_dt.feature_importances_})
df = df.sort_values(axis=0,by='weights',ascending=False)
print(df)


print(clf_dt.classes_)

"""
import io
import pydot

def show_tree(decisionTree, file_path):
    dotfile = io.StringIO()
    tree.export_graphviz(decisionTree, out_file=dotfile)
    pydot.graph_from_dot_data(dotfile.getvalue()).write_png(file_path)
    i = misc.imread(file_path)
    plt.imshow(i)

# To use it
show_tree(clf_dt, 'test.png')
"""
print(clf_dt.max_features_)
print(clf_dt.n_classes_)
print(clf_dt.n_features_)
print(clf_dt.n_outputs_)

0.525793650794
                Features   weights
16         school civics  0.438607
14   school architecture  0.220086
10        condit chicago  0.104986
7         chicago school  0.070442
17    university chicago  0.060124
3          art institute  0.050428
0   architecture chicago  0.018151
13              new york  0.013355
6            carl condit  0.012502
2       armour institute  0.011319
12  institute technology  0.000000
15        school chicago  0.000000
9    civics philanthropy  0.000000
11      institute armour  0.000000
1   architecture history  0.000000
8     chicago university  0.000000
5       building chicago  0.000000
4             art school  0.000000
18   university illinois  0.000000
['CPS' 'CS_Armour' 'CS_Sullivan' 'CS_Tallmadge' 'CS_civics']
19
5
19
1


In [34]:
#RandomForestClassifier - Ensemble Learning
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators=10,criterion="gini", max_features=None, max_depth=None, \
                                min_samples_split=2, min_samples_leaf=1, max_leaf_nodes=None, \
                                class_weight="balanced", random_state=123)

rf_acc = cross_val_score(clf_rf,train,Y,scoring="accuracy",cv=6)
#print(acc)
print(np.mean(rf_acc))

clf_rf.fit(train, Y)

#print(len(clf_dt.feature_importances_))
#print(clf_dt.feature_importances_)
#print(clf_dt.tree_)
#tree.export_graphviz(clf_dt,out_file='tree.dot')
df = pd.DataFrame(data= {"Features" : tfidf.get_feature_names(), "weights" : clf_rf.feature_importances_})
df = df.sort_values(axis=0,by='weights',ascending=False)
print(df)

0.485730097914
                Features   weights
9    civics philanthropy  0.248955
16         school civics  0.171331
14   school architecture  0.156470
7         chicago school  0.135597
10        condit chicago  0.097714
17    university chicago  0.056142
3          art institute  0.047892
6            carl condit  0.024598
13              new york  0.024013
0   architecture chicago  0.010382
8     chicago university  0.009440
5       building chicago  0.005682
12  institute technology  0.004594
18   university illinois  0.002768
2       armour institute  0.002314
15        school chicago  0.002109
1   architecture history  0.000000
11      institute armour  0.000000
4             art school  0.000000


In [42]:
#Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression

#Cs = [0.001,0.01,0.1,1,10,100,1000]
Cs = [100]

#CW = ["balanced",None]
CW = ["balanced"]

#ITR = [50,100,200,300,400]
ITR = [100]

for c in Cs:
    for cw in CW:
        for itr in ITR:
            print("C : ",c,", class_weight : ",cw,", max_iter : ",itr)
            clf_lr = LogisticRegression(class_weight=cw,C=c,max_iter=itr,random_state=123)
            acc = cross_val_score(clf_lr,train,Y,scoring="accuracy",cv=cv)
            #print(acc)
            print(np.mean(acc))

clf_lr.fit(train,Y)

print(clf_lr.classes_)

print(len(clf_lr.coef_))

#print(clf_lr.coef_)
for i, cls in enumerate(clf_lr.classes_):
    print("\nFeature weights for class : ",cls,"\n")
    df = pd.DataFrame(data= {"Features" : tfidf.get_feature_names(), "weights" : clf_lr.coef_[i]})
    df = df.sort_values(axis=0,by='weights',ascending=False)
    print(df)
    
y = np.array(Y)
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=1234)
preds = []
truths = []
for tr, tst in kfold.split(train):
    #print('%d training instances and %d testing instances' %(len(tr), len(test)))
    clf_lr_2 = LogisticRegression(class_weight="balanced",C=100,max_iter=100,random_state=123)
    clf_lr_2.fit(train[tr], y[tr])
    Y_pred = clf_lr_2.predict(train[tst])
    preds.extend(Y_pred)
    truths.extend(y[tst])
    acc = accuracy_score(y[tst], Y_pred)
    print(acc)
c_m = confusion_matrix(truths, preds,labels=clf_lr_2.classes_)
cm_df = pd.DataFrame(c_m,index=clf_lr.classes_,columns=clf_lr_2.classes_)
print(cm_df)
#print(confusion_matrix(y[tst], Y_pred, clf_lr.classes_))


C :  100 , class_weight :  balanced , max_iter :  100
0.597392647675
['CPS' 'CS_Armour' 'CS_Sullivan' 'CS_Tallmadge' 'CS_civics']
5

Feature weights for class :  CPS 

                Features   weights
17    university chicago  6.543038
7         chicago school -1.228203
11      institute armour -2.949346
4             art school -3.346784
18   university illinois -3.898123
1   architecture history -4.236043
12  institute technology -4.427742
15        school chicago -4.626057
5       building chicago -4.774758
2       armour institute -4.924696
9    civics philanthropy -5.015518
16         school civics -5.097962
6            carl condit -5.466402
13              new york -5.602438
3          art institute -5.696847
10        condit chicago -5.882224
0   architecture chicago -6.456935
8     chicago university -8.620575
14   school architecture -9.977515

Feature weights for class :  CS_Armour 

                Features    weights
3          art institute  11.122801
18   university il

In [36]:
#Support Vector Machine Classifier
from sklearn.svm import SVC,LinearSVC

clf_svc = SVC(C=10,random_state=123,class_weight="balanced")
acc = cross_val_score(clf_svc,train,Y,scoring="accuracy",cv=cv)
print(acc)
print(np.mean(acc))

clf_svc.fit(train,Y)

for c in Cs:
    for cw in CW:
        for itr in ITR:
            print("C : ",c,", class_weight : ",cw,", max_iter : ",itr)
            clf_linear_svc = LinearSVC(C=c,random_state=123,class_weight=cw,max_iter=itr)
            acc = cross_val_score(clf_linear_svc,train,Y,scoring="accuracy",cv=cv)
            #print(acc)
            print(np.mean(acc))

clf_linear_svc.fit(train,Y)

for i, cls in enumerate(clf_linear_svc.classes_):
    print("\nFeature weights for class : ",cls,"\n")
    df = pd.DataFrame(data= {"Features" : tfidf.get_feature_names(), "weights" : clf_linear_svc.coef_[i]})
    df = df.sort_values(axis=0,by='weights',ascending=False)
    print(df)

[ 0.54285714  0.33333333  0.51515152  0.5625      0.5483871 ]
0.500445817623
C :  100 , class_weight :  balanced , max_iter :  100
0.512377635805

Feature weights for class :  CPS 

                Features   weights
17    university chicago  2.007263
7         chicago school -0.555690
18   university illinois -0.987943
4             art school -1.084771
2       armour institute -1.104931
12  institute technology -1.129908
11      institute armour -1.201425
1   architecture history -1.249302
5       building chicago -1.295243
15        school chicago -1.339135
0   architecture chicago -1.366917
13              new york -1.411512
3          art institute -1.562921
6            carl condit -1.577088
10        condit chicago -1.580962
14   school architecture -1.777516
9    civics philanthropy -1.805396
16         school civics -2.039339
8     chicago university -4.272733

Feature weights for class :  CS_Armour 

                Features   weights
3          art institute  5.128076
12  in

In [37]:
#Neural Network Classifier
from sklearn.neural_network import MLPClassifier
"""
act = ["logistic","tanh","relu"]
hl = [(5,5),(5,10),(10,10),(10,15),(15,15),(15,20),(20,20)]
lr = ["constant","adaptive"]
erly_stop = [True,False]
shuffle = [True,False]
solver = ["lbfgs","sgd"]
alpha = [0.01,0.1,1,10,100]

for a in act:
    for h in hl:
        for l in lr:
            for e_s in erly_stop:
                for s in shuffle:
                    for m in ITR:
                        for so in solver:
                            for al in alpha:
                                print("Activation : ",a," hidden layers : ",h," learning rate : ",l," early_stopping : ",e_s," Shuffle : ",s," max_iter : ",m," solver :",so," Alpha : ",a)
                                clf_NN = MLPClassifier(hidden_layer_sizes=h,activation=a,solver=so,alpha=al,learning_rate=l,max_iter=m,shuffle=s,early_stopping=e_s,random_state=123)
                                acc = cross_val_score(clf_NN,train,Y,scoring="accuracy",cv=cv)
                                #print(acc)
                                print(np.mean(acc))

"""

clf_NN = MLPClassifier(hidden_layer_sizes=(5,5), activation="logistic", solver="lbfgs", alpha=0.1, \
                       learning_rate="constant", max_iter=300, shuffle=False, early_stopping=True, \
                       random_state=123)
NN_acc = cross_val_score(clf_NN,train,Y,scoring="accuracy",cv=cv)
#print(acc)
print(np.mean(NN_acc))

clf_NN.fit(train,Y)
print(len(clf_NN.coefs_))
#print(clf_NN.coefs_)
print(clf_NN.classes_)

"""
for i, cls in enumerate(clf_NN.classes_):
    print("\nFeature weights for class : ",cls,"\n")
    df = pd.DataFrame(data= {"Features" : tfidf.get_feature_names(), "weights" : clf_NN.coefs_[i]})
    df = df.sort_values(axis=0,by='weights',ascending=False)
    print(df)
"""
"""
for i, layer_weights in enumerate(clf_NN.coefs_):
    print("\nLayer : ",i)
    for j, feature in enumerate(tfidf.get_feature_names()):
        print("\nFeature : ",feature)
        print("\nCorresponding weights in layer(Note : The ith element in the list represents the weight matrix corresponding to layer i.) : ")
        print(layer_weights[j])
"""

for j, feature in enumerate(tfidf.get_feature_names()):
    print("\nFeature : ",feature)  
    print(clf_NN.coefs_[0][j])
    """
    for i, layer_weights in enumerate(clf_NN.coefs_[0]):
        print("Layer : ",i)
        #print("\nCorresponding weights in layer(Note : The ith element in the list represents the weight matrix corresponding to layer i.) : ")
        print(layer_weights[j])
    """

0.5856177559
3
['CPS' 'CS_Armour' 'CS_Sullivan' 'CS_Tallmadge' 'CS_civics']

Feature :  architecture chicago
[-1.53691571  1.45441258 -1.51736341  0.2290978  -0.36773608]

Feature :  architecture history
[ 0.57959495 -0.39637731 -1.17590027  0.68815764  0.45012884]

Feature :  armour institute
[-1.1186382   0.96798982  0.25163228  0.43103887 -1.03340006]

Feature :  art institute
[-1.55980857  1.24359299  0.79481153  0.8712381  -2.00399428]

Feature :  art school
[-0.9023666   0.73086539  0.26120615  0.32228821 -0.87831055]

Feature :  building chicago
[-0.80084189  1.00784536 -1.47961325 -0.4274138   1.04481462]

Feature :  carl condit
[ 1.49415374 -1.19292061 -1.66901669  1.34629948  0.66820291]

Feature :  chicago school
[-0.17671396  0.45755029 -0.566293   -1.54753195  2.79458574]

Feature :  chicago university
[-0.59882497  0.65774113 -2.16042192  1.70237522 -0.80136264]

Feature :  civics philanthropy
[ 0.9139676  -1.0533576   0.95839608  1.79374211 -1.87080304]

Feature :  condi

In [38]:
testData = []
#list of all snippet files in the test data
file_list = [f for f in listdir(testSnippetFolder) if not f.startswith('.')]

for file in file_list:
    filePath = testSnippetFolder + "/" + file
    f = open(filePath,'r',encoding='utf-8')
    data = f.readlines()
    textSnippet = ''.join(data)
    textSnippet = textSnippet.replace("\n","")
    for p in puncs:
        if p in textSnippet:
            textSnippet = textSnippet.replace(p,"")
    testData.append(textSnippet)
    
#use same vectorizer used to train data
test = tfidf.transform(testData)


In [39]:
#which classifier to use for classification of the test data?
Y_test_lr = clf_lr.predict(test)

"""
Y_test_svc = clf_svc.predict(test)

Y_test_linear_svc = clf_linear_svc.predict(test)

Y_test_nn = clf_NN.predict(test)
"""


print(len(file_list))
print(len(Y_test_lr))

#final list containing of (snippet file name, predicted class label) 
final_list = []
l = len(file_list)
for i in range(l):
    final_list.append((file_list[i],Y_test_lr[i]))
print(final_list)

#dict of string(key) : list (value)
#class label as key, and list of snippets classified into this label as values
d = dict()
for t in final_list:
    if t[1] not in d.keys():
        d[t[1]] = [t[0]]
    else:
        d[t[1]].append(t[0])

for key, values in d.items():
    with open(folderToSaveResultsFile + "/" + key + ".txt","a") as out_file:
        for item in values :
            out_file.write(item+"\n")

#print(d)
print("Done")

35
35
[('3102088__1.txt', 'CS_Armour'), ('hvd.32044039365119__10.txt', 'CS_Armour'), ('468039__33.txt', 'CS_Tallmadge'), ('nnc1.cu04264290__1.txt', 'CS_Armour'), ('1425278__2.txt', 'CS_Tallmadge'), ('mdp.39015031294864__1.txt', 'CS_Tallmadge'), ('774830__1.txt', 'CS_Armour'), ('hvd.32044039365119__2.txt', 'CS_civics'), ('njp.32101059234532__1.txt', 'CS_Armour'), ('1997_1047944__2.txt', 'CS_Sullivan'), ('25505243__2.txt', 'CS_Armour'), ('nnc1.ar53666712__1.txt', 'CS_Tallmadge'), ('nyp.33433071108827__4.txt', 'CS_Tallmadge'), ('1991_990613__2.txt', 'CS_Sullivan'), ('1959_1424016__1.txt', 'CS_Tallmadge'), ('773093__1.txt', 'CS_Armour'), ('mdp.39015082311138__1.txt', 'CS_Tallmadge'), ('hvd.32044039365119__9.txt', 'CS_Armour'), ('1424869__6.txt', 'CS_Tallmadge'), ('mdp.39015059791270__1.txt', 'CS_Tallmadge'), ('4117456__1.txt', 'CS_Armour'), ('ncs1.ark+=13960=t45q5jt4w__1.txt', 'CS_Tallmadge'), ('nyp.33433060463365__1.txt', 'CS_Tallmadge'), ('4102638__1.txt', 'CS_Armour'), ('mdp.39015046781

In [40]:
#try clustering techniques

In [41]:
#word2vec model